# 🌧️ Windshield Rain Generation V40 for DriveDeRain
## Realistic Water Droplets + Trails on Windshield

**Features:**
- ✅ NO falling rain streaks - Only windshield water effects
- ✅ Ice-like glass droplets (transparent + reflective)
- ✅ Smooth irregular water trails (45% opacity)
- ✅ Natural curved flow (not straight rods)
- ✅ Optimized trail count (15/18/27 for light/medium/heavy)

**Expected Time:** ~30-45 minutes for 1,550 images

---
## 📋 SECTION 1: Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

print("✓ Google Drive mounted successfully!")

---
## ⚙️ SECTION 2: Configuration

In [ ]:
# ============================================================================
# CONFIGURATION - MODIFY THESE SETTINGS
# ============================================================================

# Rain Intensity Distribution
INTENSITY_DISTRIBUTION = {
    "light": 0.40,   # 40% light rain
    "medium": 0.40,  # 40% medium rain
    "heavy": 0.20,   # 20% heavy rain
}

# Test Mode (process only 5 images per split for validation)
TEST_MODE = True  # Set to False for full dataset generation
MAX_IMAGES_TEST = 5

# Paths - MODIFY TO MATCH YOUR GOOGLE DRIVE STRUCTURE
BASE_DIR = "/content/drive/MyDrive/comp4471/project"

INPUT_DIRS = {
    "train": f"{BASE_DIR}/data/train/clear",
    "val": f"{BASE_DIR}/data/val/clear",
    "test": f"{BASE_DIR}/data/test/clear",
}

OUTPUT_DIRS = {
    "train": f"{BASE_DIR}/data/train/windshield_rainy_v40",
    "val": f"{BASE_DIR}/data/val/windshield_rainy_v40",
    "test": f"{BASE_DIR}/data/test/windshield_rainy_v40",
}

# Display configuration
print("=" * 70)
print("CONFIGURATION LOADED")
print("=" * 70)
print(f"Test Mode: {TEST_MODE}")
print(f"Base Directory: {BASE_DIR}")
print("Intensity Distribution:")
for intensity, ratio in INTENSITY_DISTRIBUTION.items():
    print(f"  - {intensity}: {ratio*100:.0f}%")
print("=" * 70)

---
## 🔧 SECTION 3: Import Libraries

In [ ]:
import cv2
import numpy as np
import random
from pathlib import Path
from tqdm.notebook import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

---
## 🎨 SECTION 4: V40 Windshield Rain Generator Class

In [ ]:
class WindshieldRainGeneratorV40:
    """
    V40: Windshield-only rain with optimized droplets + trails.
    - NO falling rain streaks
    - Ice-like glass droplets (approved)
    - Smooth irregular water trails (45% opacity)
    - Trail count: 15/18/27 for light/medium/heavy
    """

    def __init__(self, rain_intensity="medium", random_seed=42):
        random.seed(random_seed)
        np.random.seed(random_seed)

        self.presets = {
            "light": {
                "droplet_count": 75,
                "droplet_size_range": (4, 16),
                "streak_count_ratio": 0.2,  # 15 trails
                "streak_width_range": (3, 8),
            },
            "medium": {
                "droplet_count": 75,
                "droplet_size_range": (6, 24),
                "streak_count_ratio": 0.25,  # 18 trails
                "streak_width_range": (4, 10),
            },
            "heavy": {
                "droplet_count": 150,
                "droplet_size_range": (8, 32),
                "streak_count_ratio": 0.18,  # 27 trails
                "streak_width_range": (5, 12),
            },
        }

        self.params = self.presets.get(rain_intensity, self.presets["medium"])
        self.intensity = rain_intensity

    def generate_windshield_effects(self, image):
        """Generate windshield droplets + water trails."""
        h, w = image.shape[:2]
        image_float = image.astype(np.float32)
        result = image_float.copy()

        num_droplets = self.params["droplet_count"]
        num_streaks = int(num_droplets * self.params["streak_count_ratio"])

        # Generate water trails
        for _ in range(num_streaks):
            x = random.randint(0, w - 1)
            y_start = random.randint(0, int(h * 0.6))

            base_width = random.randint(*self.params["streak_width_range"])
            streak_length = random.randint(int(h * 0.2), int(h * 0.6))

            x_drift = random.randint(-15, 15)
            y_end = min(h - 1, y_start + streak_length)
            x_end = np.clip(x + x_drift, 0, w - 1)

            num_points = max(abs(y_end - y_start), 100)

            for i in range(num_points):
                t = i / num_points

                # Irregular wave
                wave_amplitude = random.uniform(3, 6)
                wave_frequency = random.uniform(0.12, 0.25)
                wave_offset = (
                    np.sin(t * np.pi * wave_frequency * num_points / 10)
                    * wave_amplitude
                )

                curr_x = int(x + t * (x_end - x) + wave_offset)
                curr_y = int(y_start + t * (y_end - y_start))

                width_variation = random.uniform(0.85, 1.15)
                curr_width = max(2, int(base_width * width_variation))

                curr_x = np.clip(curr_x, 0, w - 1)

                if 0 <= curr_y < h and 0 <= curr_x < w:
                    y1, y2 = (
                        max(0, curr_y - curr_width - 2),
                        min(h, curr_y + curr_width + 2),
                    )
                    x1, x2 = (
                        max(0, curr_x - curr_width - 2),
                        min(w, curr_x + curr_width + 2),
                    )

                    if y2 - y1 > 0 and x2 - x1 > 0:
                        region = result[y1:y2, x1:x2].copy()
                        region_h, region_w = region.shape[:2]

                        cy, cx = curr_y - y1, curr_x - x1
                        yy, xx = np.ogrid[:region_h, :region_w]
                        distances = np.sqrt((xx - cx) ** 2 + (yy - cy) ** 2)
                        blob_mask = (distances <= curr_width).astype(np.float32)

                        blob_mask = cv2.GaussianBlur(blob_mask, (11, 11), 3.0)
                        blob_mask = np.clip(blob_mask, 0, 1)

                        # Apply trail effects with 45% opacity
                        for c in range(3):
                            center_mask = (distances <= curr_width * 0.85).astype(
                                np.float32
                            )
                            center_mask = cv2.GaussianBlur(center_mask, (9, 9), 2.5)

                            opacity = 0.45
                            refraction_amount = center_mask * 3
                            region[:, :, c] = region[:, :, c] * (
                                1 - center_mask * opacity
                            ) + (region[:, :, c] + refraction_amount) * (
                                center_mask * opacity
                            )

                            thin_edge = (distances > curr_width - 2) & (
                                distances <= curr_width + 2
                            )
                            thin_edge = thin_edge.astype(np.float32)
                            thin_edge = cv2.GaussianBlur(thin_edge, (5, 5), 1.0)
                            region[:, :, c] = region[:, :, c] * (
                                1 - thin_edge * 0.03 * opacity
                            )

                            edge_mask = (
                                cv2.dilate(blob_mask, np.ones((3, 3))) - blob_mask
                            )
                            region[:, :, c] += edge_mask * 10 * opacity

                        result[y1:y2, x1:x2] = region

        # Generate droplets (grid-based distribution)
        grid_size = int(np.sqrt(num_droplets) * 1.5)
        cell_h = h / grid_size
        cell_w = w / grid_size

        droplets_per_cell = max(1, num_droplets // (grid_size * grid_size))

        for grid_y in range(grid_size):
            for grid_x in range(grid_size):
                for _ in range(droplets_per_cell):
                    x = int(grid_x * cell_w + random.uniform(0, cell_w))
                    y = int(grid_y * cell_h + random.uniform(0, cell_h))

                    x = np.clip(x, 0, w - 1)
                    y = np.clip(y, 0, h - 1)

                    size = random.randint(*self.params["droplet_size_range"])

                    y1, y2 = max(0, y - size - 2), min(h, y + size + 2)
                    x1, x2 = max(0, x - size - 2), min(w, x + size + 2)

                    if y2 - y1 > 0 and x2 - x1 > 0:
                        region = result[y1:y2, x1:x2].copy()
                        region_h, region_w = region.shape[:2]

                        cy, cx = y - y1, x - x1
                        yy, xx = np.ogrid[:region_h, :region_w]

                        # Irregular elliptical droplet
                        ellipse_ratio_x = random.uniform(0.7, 1.3)
                        ellipse_ratio_y = random.uniform(0.7, 1.3)
                        rotation = random.uniform(0, 180)

                        angle_rad = np.deg2rad(rotation)
                        cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)

                        xx_rot = (xx - cx) * cos_a - (yy - cy) * sin_a
                        yy_rot = (xx - cx) * sin_a + (yy - cy) * cos_a

                        ellipse_dist = np.sqrt(
                            (xx_rot / ellipse_ratio_x) ** 2
                            + (yy_rot / ellipse_ratio_y) ** 2
                        )

                        noise = np.random.normal(0, size * 0.05, ellipse_dist.shape)
                        ellipse_dist = ellipse_dist + noise

                        droplet_mask = (ellipse_dist <= size).astype(np.float32)
                        droplet_mask = cv2.GaussianBlur(droplet_mask, (3, 3), 0.5)
                        droplet_mask = np.clip(droplet_mask, 0, 1)

                        highlight_mask = (
                            (ellipse_dist <= size * 0.5)
                            & (yy < cy - size * 0.1)
                            & (xx < cx)
                        ).astype(np.float32)

                        # Ice-like glass droplet effects
                        for c in range(3):
                            center_mask = (ellipse_dist <= size * 0.8).astype(
                                np.float32
                            )
                            center_mask = cv2.GaussianBlur(center_mask, (7, 7), 1.5)
                            region[:, :, c] += center_mask * 8

                            thin_edge = (ellipse_dist > size - 1.5) & (
                                ellipse_dist <= size + 1.5
                            )
                            thin_edge = thin_edge.astype(np.float32)
                            region[:, :, c] = region[:, :, c] * (
                                1 - thin_edge * 0.07
                            )

                            region[:, :, c] += highlight_mask * 65

                        result[y1:y2, x1:x2] = region

        result = np.clip(result, 0, 255)
        return result

    def add_realistic_rain(self, image):
        """Main pipeline - ONLY windshield effects."""
        h, w = image.shape[:2]

        # Apply windshield droplets + trails
        rainy = self.generate_windshield_effects(image)

        # Add subtle noise
        noise = np.random.normal(0, 1, rainy.shape)
        rainy = rainy + noise

        rainy = np.clip(rainy, 0, 255).astype(np.uint8)
        rainy = cv2.GaussianBlur(rainy, (3, 3), 0)

        # Return dummy depth map for compatibility
        depth = np.zeros((h, w), dtype=np.float32)
        return rainy, depth


print("✓ WindshieldRainGeneratorV40 class loaded")

---
## 🔍 SECTION 5: Verify Dataset Structure

In [ ]:
print("=" * 70)
print("VERIFYING DATASET STRUCTURE")
print("=" * 70)

total_images = 0

for split, input_dir in INPUT_DIRS.items():
    if os.path.exists(input_dir):
        image_files = list(Path(input_dir).glob("*.jpg"))
        count = len(image_files)
        count_to_process = min(count, MAX_IMAGES_TEST) if TEST_MODE else count
        total_images += count_to_process
        print(f"✓ {split.upper()}: {count} images found, will process {count_to_process}")
    else:
        print(f"❌ {split.upper()}: Directory not found at {input_dir}")

print(f"\nTOTAL IMAGES TO PROCESS: {total_images}")

if total_images > 0:
    estimated_seconds_per_image = 1.5  # V40 is fast (no depth models)
    estimated_minutes = (total_images * estimated_seconds_per_image) / 60
    print(f"⏱️  Estimated time: {estimated_minutes:.1f} minutes")

---
## 🎨 SECTION 6: Demo Test (Single Image)

In [ ]:
import matplotlib.pyplot as plt

print("=" * 70)
print("DEMO: Testing on Single Image")
print("=" * 70)

# Find first available image
demo_image_path = None
for split, input_dir in INPUT_DIRS.items():
    if os.path.exists(input_dir):
        images = list(Path(input_dir).glob("*.jpg"))
        if images:
            demo_image_path = str(images[0])
            break

if demo_image_path:
    print(f"📸 Demo image: {Path(demo_image_path).name}")

    image = cv2.imread(demo_image_path)
    print(f"✓ Loaded: {image.shape[1]}x{image.shape[0]} pixels")

    generator = WindshieldRainGeneratorV40(
        rain_intensity="medium",
        random_seed=42
    )

    rainy_image, _ = generator.add_realistic_rain(image)

    # Display
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Original (Clear)", fontsize=14, fontweight="bold")
    axes[0].axis("off")

    axes[1].imshow(cv2.cvtColor(rainy_image, cv2.COLOR_BGR2RGB))
    axes[1].set_title("Windshield Rain V40\n(Droplets + Trails)", fontsize=14, fontweight="bold")
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()

    print("\n💡 Review quality above before proceeding!")
else:
    print("❌ No images found for demo")

---
## 🚀 SECTION 7: Generate Windshield Rain Dataset

In [ ]:
print("=" * 70)
print("GENERATING WINDSHIELD RAIN DATASET V40")
print("=" * 70)

# Prepare intensity sequence
def get_intensity_sequence(total_images, distribution):
    """Generate randomized intensity sequence."""
    intensities = []
    for intensity, ratio in distribution.items():
        count = int(total_images * ratio)
        intensities.extend([intensity] * count)
    
    # Fill remaining with medium
    while len(intensities) < total_images:
        intensities.append("medium")
    
    random.shuffle(intensities)
    return intensities

start_time = time.time()
total_processed = 0

for split, input_dir in INPUT_DIRS.items():
    if not os.path.exists(input_dir):
        continue

    output_dir = OUTPUT_DIRS[split]
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    print(f"\n{'─' * 70}")
    print(f"Processing {split.upper()} split")
    print(f"{'─' * 70}")

    image_files = list(Path(input_dir).glob("*.jpg"))
    if TEST_MODE:
        image_files = image_files[:MAX_IMAGES_TEST]

    # Generate intensity sequence
    intensities = get_intensity_sequence(len(image_files), INTENSITY_DISTRIBUTION)

    for idx, image_path in enumerate(tqdm(image_files, desc=f"  {split}")):
        try:
            # Load image
            image = cv2.imread(str(image_path))
            if image is None:
                print(f"  ⚠️  Failed to load: {image_path.name}")
                continue

            # Generate with assigned intensity
            intensity = intensities[idx]
            generator = WindshieldRainGeneratorV40(
                rain_intensity=intensity,
                random_seed=42 + idx
            )

            rainy_image, _ = generator.add_realistic_rain(image)

            # Save
            output_path = Path(output_dir) / image_path.name
            cv2.imwrite(str(output_path), rainy_image)

            total_processed += 1

        except Exception as e:
            print(f"  ❌ Error processing {image_path.name}: {e}")

    print(f"\n✓ {split.upper()} complete: {len(image_files)}/{len(image_files)} images")

elapsed_time = time.time() - start_time

print("\n" + "=" * 70)
print("✅ DATASET GENERATION COMPLETE!")
print("=" * 70)
print(f"Total images processed: {total_processed}")
print(f"Total time: {elapsed_time / 60:.1f} minutes")
print(f"Average time per image: {elapsed_time / total_processed:.2f} seconds")
print("\nOutput directories:")
for split, output_dir in OUTPUT_DIRS.items():
    if os.path.exists(output_dir):
        count = len(list(Path(output_dir).glob("*.jpg")))
        print(f"  - {split}: {output_dir} ({count} images)")
print("=" * 70)

---
## 📸 SECTION 8: Visualization - Compare Results

In [ ]:
import matplotlib.pyplot as plt
import random as rand

print("=" * 70)
print("VISUALIZING RESULTS - Comparing Clear vs Rainy")
print("=" * 70)

# Collect all available pairs across all splits
all_pairs = []

for split in ["train", "val", "test"]:
    input_dir = INPUT_DIRS[split]
    output_dir = OUTPUT_DIRS[split]
    
    if not os.path.exists(input_dir) or not os.path.exists(output_dir):
        continue
    
    # Get list of output (rainy) images
    output_images = list(Path(output_dir).glob("*.jpg"))
    
    for rainy_path in output_images:
        clear_path = Path(input_dir) / rainy_path.name
        if clear_path.exists():
            all_pairs.append((str(clear_path), str(rainy_path), split))

if len(all_pairs) == 0:
    print("\n❌ No image pairs found for visualization.")
    print("   Make sure Section 7 (dataset generation) has completed successfully.")
else:
    # Randomly sample pairs for visualization
    num_samples = min(6, len(all_pairs))  # Show up to 6 pairs
    sample_pairs = rand.sample(all_pairs, num_samples)
    
    print(f"\n📸 Showing {num_samples} random samples from generated dataset\n")
    
    # Create grid visualization
    rows = (num_samples + 1) // 2  # 2 columns
    fig, axes = plt.subplots(rows, 4, figsize=(20, rows * 5))
    
    # Flatten axes array for easier indexing
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (clear_path, rainy_path, split) in enumerate(sample_pairs):
        row = idx // 2
        col_offset = (idx % 2) * 2
        
        # Load images
        clear_img = cv2.imread(clear_path)
        rainy_img = cv2.imread(rainy_path)
        
        if clear_img is None or rainy_img is None:
            continue
        
        # Convert BGR to RGB
        clear_rgb = cv2.cvtColor(clear_img, cv2.COLOR_BGR2RGB)
        rainy_rgb = cv2.cvtColor(rainy_img, cv2.COLOR_BGR2RGB)
        
        # Display clear image
        axes[row, col_offset].imshow(clear_rgb)
        axes[row, col_offset].set_title(
            f"Clear ({split})\n{Path(clear_path).name}",
            fontsize=10,
            fontweight="bold"
        )
        axes[row, col_offset].axis("off")
        
        # Display rainy image
        axes[row, col_offset + 1].imshow(rainy_rgb)
        axes[row, col_offset + 1].set_title(
            f"Windshield Rain V40\n{Path(rainy_path).name}",
            fontsize=10,
            fontweight="bold"
        )
        axes[row, col_offset + 1].axis("off")
    
    # Hide unused subplots
    for idx in range(num_samples, rows * 2):
        row = idx // 2
        col_offset = (idx % 2) * 2
        axes[row, col_offset].axis("off")
        axes[row, col_offset + 1].axis("off")
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "=" * 70)
    print("✓ Visualization complete!")
    print(f"✓ Displayed {num_samples} sample pairs")
    print(f"✓ Total pairs available: {len(all_pairs)}")
    print("=" * 70)

---
## 📊 SECTION 9: Summary Statistics

In [ ]:
print("=" * 70)
print("DATASET SUMMARY")
print("=" * 70)

total_train = len(list(Path(OUTPUT_DIRS["train"]).glob("*.jpg"))) if os.path.exists(OUTPUT_DIRS["train"]) else 0
total_val = len(list(Path(OUTPUT_DIRS["val"]).glob("*.jpg"))) if os.path.exists(OUTPUT_DIRS["val"]) else 0
total_test = len(list(Path(OUTPUT_DIRS["test"]).glob("*.jpg"))) if os.path.exists(OUTPUT_DIRS["test"]) else 0
total_all = total_train + total_val + total_test

print(f"Train: {total_train} images")
print(f"Val:   {total_val} images")
print(f"Test:  {total_test} images")
print(f"\nTOTAL: {total_all} images")
print("\nIntensity distribution (target):")
for intensity, ratio in INTENSITY_DISTRIBUTION.items():
    expected_count = int(total_all * ratio)
    print(f"  - {intensity}: ~{expected_count} images ({ratio*100:.0f}%)")
print("=" * 70)

print("\n✅ Ready for training!")
print("\nNext steps:")
print("1. Verify image quality by spot-checking output directories")
print("2. Update training pipeline to use windshield_rainy_v40 folders")
print("3. Train your derain model with this synthetic paired dataset")

---
## 📦 SECTION 10: Zip Dataset for Download/Backup

In [ ]:
import shutil
from datetime import datetime

print("=" * 70)
print("CREATING ZIP ARCHIVE OF WINDSHIELD RAIN DATASET V40")
print("=" * 70)

# Create timestamp for unique filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"windshield_rain_v40_{timestamp}"
zip_base_path = f"{BASE_DIR}/{zip_filename}"

print(f"\n📦 Creating archive: {zip_filename}.zip")
print(f"📍 Location: {BASE_DIR}\n")
/
try:
    # Create temporary directory structure
    temp_root = f"/tmp/{zip_filename}"
    Path(temp_root).mkdir(parents=True, exist_ok=True)
    
    # Copy dataset structure
    splits_copied = []
    total_files = 0
    
    for split, output_dir in OUTPUT_DIRS.items():
        if os.path.exists(output_dir):
            files = list(Path(output_dir).glob("*.jpg"))
            if len(files) > 0:
                # Create split directory in temp
                temp_split_dir = f"{temp_root}/{split}"
                Path(temp_split_dir).mkdir(parents=True, exist_ok=True)
                
                print(f"  📁 Copying {split}: {len(files)} images...")
                
                # Copy files
                for file in files:
                    shutil.copy2(str(file), temp_split_dir)
                
                splits_copied.append(split)
                total_files += len(files)
    
    if total_files == 0:
        print("\n❌ No files found to zip. Make sure Section 7 completed successfully.")
    else:
        # Create README.txt in the archive
        readme_content = f"""Windshield Rain Dataset V40
Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

Dataset Structure:
{'─' * 50}
"""
        for split in splits_copied:
            split_dir = OUTPUT_DIRS[split]
            count = len(list(Path(split_dir).glob("*.jpg")))
            readme_content += f"{split}/\n  - {count} synthetic rainy images\n"
        
        readme_content += f"\nTotal Images: {total_files}\n"
        readme_content += f"\nIntensity Distribution:\n"
        for intensity, ratio in INTENSITY_DISTRIBUTION.items():
            expected = int(total_files * ratio)
            readme_content += f"  - {intensity}: ~{expected} images ({ratio*100:.0f}%)\n"
        
        readme_content += f"\nV40 Features:\n"
        readme_content += f"  - NO falling rain streaks (windshield only)\n"
        readme_content += f"  - Ice-like glass droplets (transparent + reflective)\n"
        readme_content += f"  - Smooth irregular water trails (45% opacity)\n"
        readme_content += f"  - Trail counts: 15/18/27 (light/medium/heavy)\n"
        readme_content += f"  - Natural curved flow (3-6px wave, ±15px drift)\n"
        readme_content += f"\nUsage:\n"
        readme_content += f"  - Pair with corresponding clear images from original dataset\n"
        readme_content += f"  - Use for training rain removal / derain models\n"
        readme_content += f"  - Filenames match original clear images for easy pairing\n"
        
        # Write README
        with open(f"{temp_root}/README.txt", "w") as f:
            f.write(readme_content)
        
        print(f"  📝 Created README.txt")
        
        # Create zip archive
        print(f"\n  🗜️  Compressing...")
        shutil.make_archive(zip_base_path, 'zip', temp_root)
        
        # Get zip file size
        zip_path = f"{zip_base_path}.zip"
        zip_size_mb = os.path.getsize(zip_path) / (1024 * 1024)
        
        # Clean up temp directory
        shutil.rmtree(temp_root)
        
        print("\n" + "=" * 70)
        print("✅ ZIP ARCHIVE CREATED SUCCESSFULLY!")
        print("=" * 70)
        print(f"📦 File: {zip_filename}.zip")
        print(f"📍 Path: {zip_path}")
        print(f"💾 Size: {zip_size_mb:.2f} MB")
        print(f"📊 Contains: {total_files} images across {len(splits_copied)} splits")
        print(f"📂 Splits: {', '.join(splits_copied)}")
        print("=" * 70)
        
        print("\n📥 To download:")
        print(f"   1. Navigate to: {BASE_DIR}")
        print(f"   2. Right-click on: {zip_filename}.zip")
        print("   3. Select 'Download'")
        print("\n💡 Or use the code below to download directly:")
        print(f"\n   from google.colab import files")
        print(f"   files.download('{zip_path}')")
        
except Exception as e:
    print(f"\n❌ Error creating zip archive: {e}")
    import traceback
    traceback.print_exc()

---
## 📥 SECTION 11: Direct Download (Optional)

In [ ]:
from google.colab import files

# Find the most recent zip file
zip_files = list(Path(BASE_DIR).glob("windshield_rain_v40_*.zip"))

if len(zip_files) == 0:
    print("❌ No zip file found. Run Section 10 first to create the archive.")
else:
    # Get most recent zip file
    latest_zip = max(zip_files, key=lambda p: p.stat().st_mtime)
    
    print("=" * 70)
    print("DOWNLOADING ZIP ARCHIVE")
    print("=" * 70)
    print(f"📦 File: {latest_zip.name}")
    print(f"💾 Size: {latest_zip.stat().st_size / (1024 * 1024):.2f} MB")
    print("\n⏳ Starting download...\n")
    
    try:
        files.download(str(latest_zip))
        print("\n✅ Download initiated! Check your browser's download folder.")
    except Exception as e:
        print(f"\n❌ Download failed: {e}")
        print("\n💡 Alternative: Navigate to Google Drive and download manually:")
        print(f"   Path: {str(latest_zip)}")